In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from PIL import Image
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras import optimizers

In [5]:
from google.colab import drive

drive.mount('/content/drive/')

path = './drive/MyDrive/CNN-Cat-Dog-Detection'

Mounted at /content/drive/


In [99]:
cat_train_path = path + '/training_set/training_set/cats/'
cat_test_path = path + '/test_set/test_set/cats/'

dog_train_path = path + '/training_set/training_set/dogs/'
dog_test_path = path +  '/test_set/test_set/dogs/'

cat_val_path = path + '/val_images/cats/'
dog_val_path = path + '/val_images/dogs/'

cat_train_img = os.listdir(cat_train_path)
dog_train_img = os.listdir(dog_train_path)[:4000]
cat_test_img = os.listdir(cat_test_path)
dog_test_img = os.listdir(dog_test_path)
cat_val_img = os.listdir(cat_val_path)
dog_val_img = os.listdir(dog_val_path)

print(len(cat_train_img))
print(len(dog_train_img))
print(len(cat_test_img))
print(len(cat_test_img))

4000
4000
1011
1011


In [11]:
img_array = []
labels = []

def img_to_array(img_data, img_path, label):

    for i in img_data:

        img = np.array(Image.open(img_path + i).resize((64, 64)))

        img_array.append(img / 255)
        labels.append(label)

img_to_array(cat_train_img, cat_train_path, 0)
img_to_array(cat_test_img, cat_test_path, 0)
img_to_array(dog_train_img, dog_train_path, 1)
img_to_array(dog_test_img, dog_test_path, 1)

len(img_array), len(labels)

(10023, 10023)

In [12]:
X = np.array(img_array)
y = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42, test_size=0.2, shuffle=True, stratify=y
)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(8018, 64, 64, 3)
(2005, 64, 64, 3)
(8018,)
(2005,)


In [67]:
def cnn_model(optimizer, epochs):

    model = Sequential()

    model.add(Conv2D(filters=64, kernel_size=3, strides=(1, 1), activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2), strides=1))

    model.add(Conv2D(filters=128, kernel_size=3, strides=(1, 1), activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2), strides=1))

    model.add(Conv2D(filters=256, kernel_size=3, strides=(1, 1), activation='relu'))

    model.add(BatchNormalization())
    model.add(Flatten())

    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))

    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])


    model.fit(X_train, y_train, epochs=epochs, batch_size=32, validation_split=0.2, callbacks=EarlyStopping(patience=3, monitor='val_loss'))

    return model

In [53]:
model_1 = cnn_model(optimizer=optimizers.Adam(learning_rate=0.000001), epochs=100)

Epoch 1/100
201/201 [==============================] - 8s 27ms/step - loss: 0.6697 - accuracy: 0.5820 - val_loss: 0.6838 - val_accuracy: 0.6359
Epoch 2/100
201/201 [==============================] - 5s 25ms/step - loss: 0.6033 - accuracy: 0.6882 - val_loss: 0.6586 - val_accuracy: 0.6390
Epoch 3/100
201/201 [==============================] - 5s 24ms/step - loss: 0.5479 - accuracy: 0.7466 - val_loss: 0.6159 - val_accuracy: 0.6721
Epoch 4/100
201/201 [==============================] - 5s 25ms/step - loss: 0.4946 - accuracy: 0.7830 - val_loss: 0.5990 - val_accuracy: 0.6758
Epoch 5/100
201/201 [==============================] - 5s 25ms/step - loss: 0.4445 - accuracy: 0.8274 - val_loss: 0.5935 - val_accuracy: 0.6895
Epoch 6/100
201/201 [==============================] - 5s 25ms/step - loss: 0.4028 - accuracy: 0.8556 - val_loss: 0.5842 - val_accuracy: 0.6970
Epoch 7/100
201/201 [==============================] - 5s 25ms/step - loss: 0.3605 - accuracy: 0.8865 - val_loss: 0.5866 - val_accuracy:

In [54]:
model_1.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 62, 62, 64)        1792      
                                                                 
 max_pooling2d_16 (MaxPooli  (None, 61, 61, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_25 (Conv2D)          (None, 59, 59, 128)       73856     
                                                                 
 max_pooling2d_17 (MaxPooli  (None, 58, 58, 128)       0         
 ng2D)                                                           
                                                                 
 conv2d_26 (Conv2D)          (None, 56, 56, 256)       295168    
                                                                 
 batch_normalization_8 (Bat  (None, 56, 56, 256)      

In [61]:
model_1.evaluate(X_test, y_test)

63/63 [==============================] - 0s 7ms/step - loss: 0.5918 - accuracy: 0.7242


[0.5918000936508179, 0.7241895198822021]

In [83]:
model_2 = cnn_model(optimizer=optimizers.RMSprop(learning_rate=0.000001), epochs=50)

Epoch 1/50
201/201 [==============================] - 7s 26ms/step - loss: 0.6851 - accuracy: 0.5566 - val_loss: 0.6911 - val_accuracy: 0.5711
Epoch 2/50
201/201 [==============================] - 5s 23ms/step - loss: 0.6662 - accuracy: 0.6099 - val_loss: 0.6846 - val_accuracy: 0.5804
Epoch 3/50
201/201 [==============================] - 5s 23ms/step - loss: 0.6520 - accuracy: 0.6360 - val_loss: 0.6706 - val_accuracy: 0.6110
Epoch 4/50
201/201 [==============================] - 5s 24ms/step - loss: 0.6392 - accuracy: 0.6603 - val_loss: 0.6573 - val_accuracy: 0.6259
Epoch 5/50
201/201 [==============================] - 5s 23ms/step - loss: 0.6274 - accuracy: 0.6743 - val_loss: 0.6488 - val_accuracy: 0.6415
Epoch 6/50
201/201 [==============================] - 5s 23ms/step - loss: 0.6167 - accuracy: 0.6911 - val_loss: 0.6422 - val_accuracy: 0.6415
Epoch 7/50
201/201 [==============================] - 5s 23ms/step - loss: 0.6063 - accuracy: 0.7025 - val_loss: 0.6363 - val_accuracy: 0.6453

In [84]:
model_2.summary()

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_72 (Conv2D)          (None, 62, 62, 64)        1792      
                                                                 
 max_pooling2d_48 (MaxPooli  (None, 61, 61, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_73 (Conv2D)          (None, 59, 59, 128)       73856     
                                                                 
 max_pooling2d_49 (MaxPooli  (None, 58, 58, 128)       0         
 ng2D)                                                           
                                                                 
 conv2d_74 (Conv2D)          (None, 56, 56, 256)       295168    
                                                                 
 batch_normalization_24 (Ba  (None, 56, 56, 256)     

In [85]:
model_2.evaluate(X_test, y_test)

63/63 [==============================] - 0s 7ms/step - loss: 0.5559 - accuracy: 0.7132


[0.555901825428009, 0.7132169604301453]

In [2]:
model_1 = load_model('best_cnn_model_with_Adam')

In [3]:
model_2 = load_model('best_cnn_model_with_RMSprop')

In [111]:
img_array = []
labels = []

def img_to_array(img_data, img_path, label):

    for i in img_data:

        img = Image.open(img_path + i).resize((64, 64))
        img = np.array(img.convert('RGB'))
        img_array.append( img / 255 )
        labels.append(label)

img_to_array(cat_val_img, cat_val_path, 0)
img_to_array(dog_val_img[:-1], dog_val_path, 1)

len(labels), len(img_array)

(12, 12)

In [113]:
X_val = np.array(img_array)
y_val = np.array(labels)

print(X_val.shape)
print(y_val.shape)

(12, 64, 64, 3)
(12,)


In [127]:
len(model_1.predict(X_val))

1/1 [==============================] - 0s 244ms/step


12

In [149]:
def test_model(model):
    y_pred = []
    for i in range(len(model.predict(X_val))):
        predict = model.predict(X_val)[i][0]

        if predict < 0.5:
            y_pred.append(0)
        elif predict > 0.5:
            y_pred.append(1)
    
    return 'accuracy_score:' + str(accuracy_score(y_val, y_pred))

In [150]:
test_model(model_1)

1/1 [==============================] - 0s 256ms/step


'accuracy_score:0.5833333333333334'

In [151]:
test_model(model_2)

1/1 [==============================] - 0s 238ms/step


'accuracy_score:0.5'